In [1]:
import numpy as np

import torch
import torch.optim as optim
import torch.nn as nn
from FairRanking.datasets.adult import Adult
from FairRanking.datasets.law import Law
from FairRanking.datasets.compas import Compas
from FairRanking.datasets.wiki import Wiki
from FairRanking.models.DebiasClassifier import DebiasClassifier
from FairRanking.TrainingFunctions.DebiasClassifierTrain import train
from FairRanking.helpers import rND_torch, nDCG_cls_no_model, auc_estimator, group_pairwise_accuracy, nDCG_cls, auc_estimator2, calc_accuracy

In [17]:
#data = Law('Gender','/Users/robert/Desktop/Bachelor/FairRanker/data')
#data = Adult('/Users/robert/Desktop/Bachelor/FairRanker/data')
data = Compas('/Users/robert/Desktop/Bachelor/FairRanker/data')
#data = Wiki()
#X_train, s_train, y_train, X_val, s_val, y_val, X_test, s_test, y_test = convert_data_to_tensors(data, build_pairs=False)
full_data = data.get_data()

/opt/anaconda3/envs/pytorchbook/lib/python3.10/site-packages/FairRanking/datasets/compas.py:25: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  self.df.loc[:, x_name] = scaler.fit_transform(self.df.loc[:, x_name])
/opt/anaconda3/envs/pytorchbook/lib/python3.10/site-packages/FairRanking/datasets/compas.py:25: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  self.df.loc[:, x_name] = scaler.fit_transform(self.df.loc[:, x_name])
/opt/anaconda3/envs/pytorchbook/lib/python3.10/site-packages/FairRanking/datasets/compas.py:25: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  self.df.loc[:, x_name] = scaler.fit_transform(self.df.loc[:, x_name])
/opt/anaconda3/envs/pytorchbook/lib/python3.10/site-packages/FairRanking/datasets/compas.py:26: FutureWarning: Passing a set as an indexer is deprecated and 

In [44]:
model = DebiasClassifier(num_features=data.get_num_features(),
                    kernel_initializer=nn.init.normal_,
                    hidden_layers=[60],
                    bias_layers=[16, 2],
                    num_relevance_classes=data.num_relevance_class)

In [53]:

model = train(model, full_data, schedule = [1,0], adv_lr = 0.01, main_lr=0.01, n_epochs = 1000, print_res=True)

0\1000	 Loss: 1.8195387125015259
100\1000	 Loss: 1.9078971147537231
200\1000	 Loss: 2.0612707138061523
300\1000	 Loss: 1.9721601009368896
400\1000	 Loss: 1.9547147750854492
500\1000	 Loss: 1.8968379497528076
600\1000	 Loss: 1.9317378997802734
700\1000	 Loss: 1.9591004848480225
800\1000	 Loss: 1.9043887853622437
900\1000	 Loss: 1.905897617340088


In [50]:
from sklearn.metrics import accuracy_score, f1_score
data_idx = 2
model.eval()
y = full_data[data_idx][1]
#y = full_data[data_idx][1].squeeze() # if Adult
#print(y)
with torch.no_grad():
    pred = model.predict_proba(full_data[data_idx][0])
    #print('GPA: ', group_pairwise_accuracy(pred, full_data[data_idx][1], full_data[data_idx][2]))
    print('AUC: ', auc_estimator2(pred, y, multiclass=True))
    pred = torch.argmax(pred, dim=1)
    print('nDCG: ', nDCG_cls(pred, y, esti=False, at=500))
    print('ACC', accuracy_score(y, pred))
    #print('AUC', auc_sklearn(pred, full_data[data_idx][1]))
    print('rND: ', rND_torch(pred, torch.argmax(full_data[data_idx][2], dim=1)))

AUC:  0.7128827316199213
nDCG:  0.6737509232462537
ACC 0.26178861788617885
rND:  0.3384815502791561


In [19]:
nn.L1Loss(reduction='mean')(pred, y)

tensor(1.8276)

In [43]:
data_idx = 0
model.eval()
with torch.no_grad():
    pred = model.predict_proba(full_data[data_idx][0])
    pred = torch.argmax(pred, dim=1)
    y = full_data[data_idx][1].squeeze()
    print(y)
    print(pred)
    print(torch.unique(pred == y, return_counts=True))
    print(len(full_data[data_idx][1]))


tensor([3., 1., 3.,  ..., 3., 1., 8.])
tensor([1, 1, 5,  ..., 4, 0, 7])
(tensor([False,  True]), tensor([2800, 1136]))
3936


In [44]:
print(full_data[0][0])

tensor([[1., 0., 1.,  ..., 0., 0., 1.],
        [1., 0., 1.,  ..., 0., 0., 1.],
        [0., 0., 8.,  ..., 1., 1., 0.],
        ...,
        [1., 0., 4.,  ..., 0., 0., 1.],
        [0., 0., 1.,  ..., 1., 1., 0.],
        [0., 1., 4.,  ..., 0., 1., 0.]])


In [16]:
for name, p in model.named_parameters():
    print(name)

feature_layers.0.weight
feature_layers.0.bias
feature_layers.1.weight
feature_layers.1.bias
debias_layers.0.weight
debias_layers.0.bias
debias_layers.1.weight
debias_layers.1.bias
debias_layers.2.weight
debias_layers.2.bias
additional_main_layers.0.weight
additional_main_layers.0.bias
additional_main_layers.1.weight
additional_main_layers.1.bias
output_layer.weight
output_layer.bias
